In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import rc
import scipy as sp
import scipy.integrate as spi
import scipy.interpolate as spp
import scipy.optimize as spo

# ---------------------------------------------------
# activate latex text rendering
# ---------------------------------------------------
#%matplotlib inline

plt.rcParams['text.usetex'] = True

#LaTex setting
plt.rcParams['text.latex.preamble']=r"\usepackage{amsmath}"
plt.rcParams['text.latex.preamble'] = r'\boldmath'
#!apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng

#rc('text', usetex=True)

# ------------------------------------------------------

# -----------------------------------------------------------------------
plt.rcParams['figure.figsize'] = (10, 7)
plt.rcParams['font.size'] = 12
#plt.rcParams['font.family'] = 'Times New Roman'


plt.rcParams['axes.labelsize'] = plt.rcParams['font.size']
plt.rcParams['axes.titlesize'] = 1.2*plt.rcParams['font.size']
#plt.rcParams['legend.fontsize'] = plt.rcParams['font.size']
plt.rcParams['xtick.labelsize'] = 1.2*plt.rcParams['font.size']
plt.rcParams['ytick.labelsize'] = 1.2*plt.rcParams['font.size']
# dots per inch: dpi
#plt.rcParams['savefig.dpi'] = 2*plt.rcParams['savefig.dpi']

plt.rcParams['xtick.major.size'] = 3
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

#legends
#plt.rcParams['legend.frameon'] = False
#plt.rcParams['legend.loc'] = 'center left'
#plt.rcParams['legend.fontsize'] = plt.rcParams['font.size']

plt.rcParams['axes.linewidth'] = 1

#border setting
#plt.gca().spines['right'].set_color('none')
#plt.gca().spines['top'].set_color('none')

#ticks position setting
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('left')

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [16]:
Type = '100sqrt(16)_8.0e-6'
Title = r'$\lambda_{_{\rm T}} = 50 \sqrt{\frac{2}{3}}$'#,~g^2 = 2.8 \times 10^{-6}$'
mu = np.loadtxt('Flq_Exp_' + 'Tmodel_' + Type + '.txt')
# mu = np.loadtxt('Flq_Exp_Mathieu_ext2' + '.txt')

# Defining Parameters (Following are in GeV)
# If the quantites you have are already dimensionless, make the appropriate changes here

lam = 50*(2/3)**0.5 #100*(1/6)**0.5               # The coupling parameter
m2 = 9.4868e+27 #1.6799e+28                    # Mass of the field squared
m = m2**0.5                        # Mass of the field
mp = 2.435e18                      # reduced Planck Mass
g2 = 1.6e-6 #2.8e-6

# Converting them into dimensionless quantities (w.r.t. m)              
m = m/mp
V0 = 4.8000e-13 #8.5000e-13                    # V_0/mp^4

# Constructing arrays
n = 250
x0 = np.linspace(0.001, 0.06, n)
k = np.linspace(0.1, 2.5, n)

q = 0.25*g2*(x0/m)**2.0
Ak = k**2.0 + 2*q

print(m)
# print(Ak)
# print(q)

4.00000843279641e-05


In [17]:
# Defining Inflationary Potential 
# Write the potential and its first, second derivatives here 

# E-model (Disable when not in use)
def V(xx, lam = lam):
    return V0*(1 - np.exp(-lam*xx))**2.0

def dV(xx, lam = lam):
    return 2.0*V0*lam*np.exp(-2.0*lam*xx)*(np.exp(lam*xx) - 1)

def d2V(xx, lam = lam):
    return -2.0*V0*(lam**2.0)*np.exp(-2.0*lam*xx)*(np.exp(lam*xx) - 2)

# T-model (Disable when not in use)
# def V(xx, lam = lam):
#     return V0*np.tanh(lam*xx)**2.0

# def dV(xx, lam = lam):
#     return 2.0*V0*lam*np.tanh(lam*xx)*np.power(np.cosh(lam*xx), -2.0)

# def d2V(xx, lam = lam):
#     return 2.0*V0*(lam**2.0)*(np.power(np.cosh(lam*xx), -4.0) - 2.0*np.power(np.tanh(lam*xx), 2.0)*np.power(np.cosh(lam*xx), -2.0))

In [18]:
# Calculating the Time-period of the Oscillation

# Finding the points where V = V(x0)
def fldvls(x0, lam):
    Vend = V(x0, lam)
    if V(-x0, lam) == Vend:
        return [-x0, x0]
    else:
        def roots(xx, x0, lam = lam):
            return V(xx, lam) - V(x0, lam)
        rts = spo.root_scalar(roots, args=(x0, lam), bracket=[-x0, 0])
        return [rts.root, x0] 
    
def timeintegrand(xx, xlim, lam = lam):
    return (2.0*V(xlim, lam) - 2.0*V(xx, lam))**(-0.5)

def TimePeriod(x0, lam = lam):
    x_minus, x_plus = fldvls(x0, lam)
    T_minus = spi.quad(timeintegrand, x_minus, 0, args = (x_minus, lam), epsabs = 10e-10)[0]
    T_plus = spi.quad(timeintegrand, 0, x_plus, args = (x_plus, lam), epsabs = 10e-10)[0]

    return 2.0*(T_minus + T_plus)

In [19]:
Y, X = np.meshgrid(k, x0)

T = TimePeriod(0.0385, lam)
t = np.linspace(0.1, T, 800000)
a = t**(2/3)

x = 0.6/a**1.5
K = [1.5/a, 3.0/a, 5.0/a, 7.0/a, 9.0/a]
# K = [1.5/a, 6.0/a, 11.0/a, 17.0/a, 23.0/a]

cmap = 'hot'
# cf = plt.contourf(X, Y, np.abs(mu), 250, cmap = cmap, vmin = -0.025, vmax = 0.195)
cf = plt.contourf(X, Y, np.abs(mu), 250, cmap = cmap, vmin = np.min(mu) - 0.015, vmax = np.max(mu) + 0.050)
for i in range(len(K)):
    plt.plot(x, K[i], color = 'white', lw = 1.25)

# plt.text(0.01957, 2.424, r'$k \propto a^{-1}$',  ha = 'center', va = 'center', color = 'white', fontsize = 18, rotation = 35)
plt.text(0.0282, 1.281, r'$k \propto a^{-1}$',  ha = 'center', va = 'center', color = 'white', fontsize = 18, rotation = 35)

# plt.annotate('', xy=(0.0151028, 1.97547), xytext=(0.0147599, 1.94545), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
# plt.annotate('', xy=(0.0173026, 1.59856), xytext=(0.0168572, 1.57109), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
# plt.annotate('', xy=(0.0191359, 1.10627), xytext=(0.0186541, 1.08756), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
# plt.annotate('', xy=(0.0195014, 0.61101), xytext=(0.0189539, 0.59955), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
# plt.annotate('', xy=(0.0195870, 0.15320), xytext=(0.0193646, 0.15205), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))

plt.annotate('', xy=(0.026252, 1.1173), xytext=(0.024878, 1.0780), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(0.029295, 0.93509), xytext=(0.028093, 0.90932), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(0.031428, 0.69999), xytext=(0.030451, 0.68532), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(0.033255, 0.43609), xytext=(0.032202, 0.42681), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(0.0341683, 0.22198), xytext=(0.0334156, 0.21874), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))

# plt.annotate(Title, ha = 'center', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.50, 1.0), xycoords='axes fraction', size = 20)
plt.xlabel(r'$\phi_0/m_p$', size = 24)
plt.ylabel(r'$k/m$', size = 24)
clb = plt.colorbar()
clb.ax.set_title(r'$\Re(\nu_k)/m$', size = 18)
plt.xlim(np.min(x0), np.max(x0))
# plt.ylim(np.min(k), 1.95)
plt.ylim(np.min(k), np.max(k))
# plt.tight_layout()
plt.show()

In [8]:
Y, X = np.meshgrid(Ak, q)

T = TimePeriod(0.0385, lam)
t = np.linspace(0.1, T, 800000)
a = t**(2/3)

x = 0.6/a**1.5
AK = [5*q, 4*q, 3*q, 2*q, 1*q]


cmap = 'hot'
# cf = plt.contourf(X, Y, np.abs(mu), 250, cmap = cmap, vmin = -0.025, vmax = 0.195)
cf = plt.contourf(X, Y, np.abs(mu), 250, cmap = cmap, vmin = np.min(mu) - 0.015, vmax = np.max(mu) + 0.050)
for i in range(len(AK)):
    plt.plot(q, AK[i], color = 'white', lw = 1.25)

# plt.text(3.092, 3.34, r'$A_k \propto q$',  ha = 'center', va = 'center', color = 'white', fontsize = 18, rotation = 30)
plt.text(5.49, 6.10, r'$A_k \propto q$',  ha = 'center', va = 'center', color = 'white', fontsize = 18, rotation = 19)

plt.annotate('', xy=(1.34446, 6.7222), xytext=(1.32061, 6.6032), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(1.56827, 6.2732), xytext=(1.54999, 6.2002), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(1.88637, 5.65912), xytext=(1.86498, 5.59506), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(2.33559, 4.6710), xytext=(2.29526, 4.5903), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))
plt.annotate('', xy=(2.6946, 2.69450), xytext=(2.6554, 2.65534), arrowprops=dict(arrowstyle='<-', lw=2, color='white'))

# plt.annotate(Title, ha = 'center', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.50, 1.0), xycoords='axes fraction', size = 20)
plt.xlabel(r'$q$', size = 24)
plt.ylabel(r'$A_k$', size = 24)
clb = plt.colorbar()
clb.ax.set_title(r'$\Re(\mu_k)/m$', size = 18)
plt.ylim(np.min(Ak), np.max(Ak))
plt.xlim(np.min(q), np.max(q))

# plt.xlim(np.min(q), 5)
# plt.ylim(0, 9)
plt.show()

In [20]:
colour ='tab:red'
Mu = spp.RegularGridInterpolator((x0, k), np.abs(mu), method = 'cubic')
MMu1 = []
MMu2 = []
for i in range(len(k)):
    MMu1 = np.append(MMu1, Mu([0.06, k[i]]))
    # MMu2 = np.append(MMu2, Mu([0.06, k[i]]))

plt.axhline(0.0, color='k', lw = 1.2)
plt.plot(k, MMu1, lw = 2.5, color=colour, label =r'$\phi_0 = 0.06 m_p$')
# plt.plot(k, MMu2, lw = 2.5, color='b', label =r'$\phi_0 = 0.06 m_p$')
plt.xlabel(r'$k/m$', size = 24)
# plt.xlabel(r'$\phi_0/m_p$', size = 24)
plt.ylabel(r'$\Re(\nu_k)/m$', size = 24)
plt.xlim(np.min(k), np.max(k))
# plt.annotate(r'$\lambda_{\rm {\tiny E}} = 50\sqrt{\frac{2}{3}}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.8525), xycoords='axes fraction', size =  25)
# plt.annotate(r'$g^2 = 1.6 \times 10^{-6}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.8525), xycoords='axes fraction', size =  25)
# plt.annotate(r'$\lambda_{\rm {\tiny T}} = 50\sqrt{\frac{2}{3}}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.8525), xycoords='axes fraction', size =  25)
plt.annotate(r'$g^2 = 8.0 \times 10^{-6}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.8525), xycoords='axes fraction', size =  25)

plt.fill_between(k, MMu1, color=colour, alpha = 0.2)
# plt.fill_between(k, MMu2, color='b', alpha = 0.2)
plt.legend(prop={'size': 22})
plt.tight_layout()

In [23]:
colour ='tab:red'
Mu = spp.RegularGridInterpolator((x0, k), np.abs(mu), method = 'cubic')
MMu1 = []
for i in range(len(k)):
    MMu1 = np.append(MMu1, Mu([x0[i], 0.7]))

plt.axhline(0.0, color='k', lw = 1.2)
plt.plot(x0, MMu1, lw = 2.5, color = colour, label =r'$k = 0.7 m$')
plt.xlabel(r'$\phi_0/m_p$', size = 24)
plt.ylabel(r'$\Re(\nu_k)/m$', size = 24)
plt.xlim(np.min(x0), np.max(x0))
# plt.annotate(r'$\lambda_{\rm {\tiny E}} = 50\sqrt{\frac{2}{3}}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.96), xycoords='axes fraction', size =  25)
# plt.annotate(r'$g^2 = 1.6 \times 10^{-6}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.8525), xycoords='axes fraction', size =  25)
# plt.annotate(r'$\lambda_{\rm {\tiny T}} = 50\sqrt{\frac{2}{3}}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.96), xycoords='axes fraction', size =  25)
# plt.annotate(r'$g^2 = 8.0 \times 10^{-6}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.8525), xycoords='axes fraction', size =  25)
plt.annotate(r'$g^2 = 8.0 \times 10^{-6}$', va = 'top', ha = 'right', bbox = {'boxstyle': 'round', 'facecolor': 'white', 'pad': 0.3}, xy=(0.97, 0.96), xycoords='axes fraction', size =  25)

plt.fill_between(x0, MMu1, color = colour, alpha = 0.2)
plt.legend(prop={'size': 22}, loc = 'upper left')
plt.tight_layout()

In [ ]:
# Historia

# E,T-models
n = 250
x0 = np.linspace(0.001, 0.06, n)
k = np.linspace(0.1, 2.5, n)

n = 250
x0 = np.linspace(0.01, 0.09, n)
k = np.linspace(0.1, 2.5, n)

n = 250
x0 = np.linspace(0.01, 0.09, n)
k = np.linspace(0.1, 2.0, n)

n = 250
x0 = np.linspace(0.001, 0.03, n)   # ext2
k = np.linspace(0.1, 3.0, n)